In [1]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from tqdm import tqdm

from PIL import Image

import csv

In [2]:
DATA_DIR = "/Volumes/SSD970/"

In [13]:
bbox_df = pd.read_csv(os.path.join(DATA_DIR, 'train_bounding_boxes.csv')).set_index('StudyInstanceUID')
bbox_df.head()

,x,y,width,height,slice_number
StudyInstanceUID,,,,,
1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,214.00000,215.48637,27.00000,43.51363,137


In [32]:
bbox_df.groupby("StudyInstanceUID").count()

,x,y,width,height,slice_number
StudyInstanceUID,,,,,
1.2.826.0.1.3680043.10051,16,16,16,16,16
1.2.826.0.1.3680043.10579,3,3,3,3,3
1.2.826.0.1.3680043.10678,62,62,62,62,62
1.2.826.0.1.3680043.10697,8,8,8,8,8
1.2.826.0.1.3680043.10732,31,31,31,31,31
...,...,...,...,...,...
1.2.826.0.1.3680043.8519,6,6,6,6,6
1.2.826.0.1.3680043.8693,6,6,6,6,6
1.2.826.0.1.3680043.9447,33,33,33,33,33


In [30]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_df_with_slices.csv')).set_index("StudyInstanceUID")
train_df.head()

,patient_overall,C1,C2,C3,C4,C5,C6,C7,num_slices
StudyInstanceUID,,,,,,,,,
1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0,243
1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0,406
1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0,385
1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0,501
1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0,199


In [25]:
bbox_df.index.unique()

Index(['1.2.826.0.1.3680043.10051', '1.2.826.0.1.3680043.10579',
       '1.2.826.0.1.3680043.10678', '1.2.826.0.1.3680043.10697',
       '1.2.826.0.1.3680043.10732', '1.2.826.0.1.3680043.10815',
       '1.2.826.0.1.3680043.11103', '1.2.826.0.1.3680043.11227',
       '1.2.826.0.1.3680043.11300', '1.2.826.0.1.3680043.11401',
       ...
       '1.2.826.0.1.3680043.8043', '1.2.826.0.1.3680043.8198',
       '1.2.826.0.1.3680043.8330', '1.2.826.0.1.3680043.8379',
       '1.2.826.0.1.3680043.8511', '1.2.826.0.1.3680043.8519',
       '1.2.826.0.1.3680043.8693', '1.2.826.0.1.3680043.9447',
       '1.2.826.0.1.3680043.9926', '1.2.826.0.1.3680043.9940'],
      dtype='object', name='StudyInstanceUID', length=235)

pick only included in the bbox dataset

In [71]:
num_slices_df = train_df.loc[list(bbox_df.index.unique()), ["num_slices"]]
num_slices_df.head()

,num_slices
StudyInstanceUID,
1.2.826.0.1.3680043.10051,272
1.2.826.0.1.3680043.10579,184
1.2.826.0.1.3680043.10678,269
1.2.826.0.1.3680043.10697,179
1.2.826.0.1.3680043.10732,218


In [36]:
bbox_df.insert(0, "num_slices", None)
bbox_df.head()

,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,
1.2.826.0.1.3680043.10051,None,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,None,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,None,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,None,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,None,214.00000,215.48637,27.00000,43.51363,137


In [72]:
num_slices_df.iloc[0]

num_slices    272
Name: 1.2.826.0.1.3680043.10051, dtype: int64

Insert num_slices value

In [73]:
for i in range(len(num_slices_df)):
    row = num_slices_df.iloc[i]

    UID = row.name
    num_slices = row.num_slices

    bbox_df.loc[UID, "num_slices"] = num_slices
bbox_df.head()

,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,
1.2.826.0.1.3680043.10051,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,272,214.00000,215.48637,27.00000,43.51363,137


In [77]:
vertical_df = pd.read_csv(os.path.join(DATA_DIR, 'meta_train_3d.csv')).set_index('UID')
print(len(vertical_df))
vertical_df.head()

2012


In [76]:
bbox_df.insert(0, "is_flip", None)
bbox_df.head()

,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,
1.2.826.0.1.3680043.10051,None,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,None,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,None,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,None,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,None,272,214.00000,215.48637,27.00000,43.51363,137


In [81]:
bbox_df = bbox_df.drop("1.2.826.0.1.3680043.22678")

In [82]:
vertical_df = vertical_df.loc[list(bbox_df.index.unique())]
len(vertical_df)

234

In [84]:
vertical_df.head()

z_spacing        0.625000
pixel_spacing    0.253906
aspect           2.461541
is_flip          0.000000
Name: 1.2.826.0.1.3680043.10051, dtype: float64

Insert is_flip

In [85]:
vertical_df[["z_spacing", "pixel_spacing", "aspect", "is_flip"]]

,z_spacing,pixel_spacing,aspect,is_flip
UID,,,,
1.2.826.0.1.3680043.10051,0.62500,0.253906,2.461541,0
1.2.826.0.1.3680043.10579,1.00000,0.328125,3.047619,0
1.2.826.0.1.3680043.10678,0.80000,0.261000,3.065134,0
1.2.826.0.1.3680043.10697,1.00000,0.267578,3.737226,0
1.2.826.0.1.3680043.10732,1.00000,0.240234,4.162602,0
...,...,...,...,...
1.2.826.0.1.3680043.8519,0.62500,0.248047,2.519684,0
1.2.826.0.1.3680043.8693,2.62604,0.488281,5.378133,0
1.2.826.0.1.3680043.9447,0.40000,0.289062,1.383784,0


In [87]:
bbox_df.insert(0, "z_spacing", None)
bbox_df.insert(0, "pixel_spacing", None)
bbox_df.insert(0, "aspect", None)
bbox_df.head()

,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,
1.2.826.0.1.3680043.10051,None,None,None,None,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,None,None,None,None,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,None,None,None,None,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,None,None,None,None,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,None,None,None,None,272,214.00000,215.48637,27.00000,43.51363,137


In [95]:
for i in range(len(vertical_df)):
    row = vertical_df.iloc[i]

    UID = row.name
    # num_slices = row.num_slices

    bbox_df.loc[UID, ["aspect", "pixel_spacing", "z_spacing", "is_flip"]] = row[["aspect", "pixel_spacing", "z_spacing", "is_flip"]].values
bbox_df.head()

/Users/longyikim/anaconda3/envs/pytorch/lib/python3.9/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,
1.2.826.0.1.3680043.10051,2.461541,0.253906,0.625,0.0,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,2.461541,0.253906,0.625,0.0,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,2.461541,0.253906,0.625,0.0,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,2.461541,0.253906,0.625,0.0,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,2.461541,0.253906,0.625,0.0,272,214.00000,215.48637,27.00000,43.51363,137


In [97]:
bbox_df.insert(0, "sagittal_index", None)
bbox_df.insert(0, "coronal_index", None)
bbox_df.insert(0, "axial_index", None)
bbox_df.insert(0, "start_slice_number", None)
bbox_df.head()

,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,
1.2.826.0.1.3680043.10051,None,None,None,2.461541,0.253906,0.625,0.0,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,None,None,None,2.461541,0.253906,0.625,0.0,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,None,None,None,2.461541,0.253906,0.625,0.0,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,None,None,None,2.461541,0.253906,0.625,0.0,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,None,None,None,2.461541,0.253906,0.625,0.0,272,214.00000,215.48637,27.00000,43.51363,137


In [99]:
bbox_df.insert(0, "start_slice_number", None)

In [100]:
bbox_df.head()

,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10051,None,None,None,None,2.461541,0.253906,0.625,0.0,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,None,None,None,None,2.461541,0.253906,0.625,0.0,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,None,None,None,None,2.461541,0.253906,0.625,0.0,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,None,None,None,None,2.461541,0.253906,0.625,0.0,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,None,None,None,None,2.461541,0.253906,0.625,0.0,272,214.00000,215.48637,27.00000,43.51363,137


In [105]:
#
def get_start_train_index(UID):
    dcm_list = glob.glob(os.path.join(DATA_DIR, 'train_images', UID) + "/*.dcm")
    dcm_list.sort(key=lambda x:int(x.split("/")[-1].replace(".dcm", "")))
    start_index = dcm_list[0].split("/")[-1].replace(".dcm", "")
    return int(start_index)
UID = bbox_df.index.unique().values[0]
get_start_train_index(UID)

1

In [106]:
for UID in tqdm(bbox_df.index.unique().values):
    start_index = get_start_train_index(UID)
    bbox_df.loc[UID, "start_slice_number"] = start_index
bbox_df.head()

  0%|          | 0/234 [00:00<?, ?it/s]/Users/longyikim/anaconda3/envs/pytorch/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
100%|██████████| 234/234 [00:00<00:00, 330.30it/s]


,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10051,1,None,None,None,2.461541,0.253906,0.625,0.0,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,1,None,None,None,2.461541,0.253906,0.625,0.0,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,1,None,None,None,2.461541,0.253906,0.625,0.0,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,1,None,None,None,2.461541,0.253906,0.625,0.0,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,1,None,None,None,2.461541,0.253906,0.625,0.0,272,214.00000,215.48637,27.00000,43.51363,137


In [124]:
bbox_df[bbox_df["start_slice_number"] > 1]

,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.24673,2,129,None,None,3.875995,0.25,0.968999,0.0,218,225.09506,112.42586,95.39163,50.61597,131
1.2.826.0.1.3680043.24673,2,130,None,None,3.875995,0.25,0.968999,0.0,218,176.42586,112.64251,160.60837,111.87425,132
1.2.826.0.1.3680043.24673,2,131,None,None,3.875995,0.25,0.968999,0.0,218,182.00000,113.39924,162.82130,113.46551,133
1.2.826.0.1.3680043.24673,2,132,None,None,3.875995,0.25,0.968999,0.0,218,180.70756,113.39924,155.29244,111.60076,134
1.2.826.0.1.3680043.24673,2,133,None,None,3.875995,0.25,0.968999,0.0,218,180.00000,113.00000,156.00000,112.00000,135
1.2.826.0.1.3680043.24673,2,134,None,None,3.875995,0.25,0.968999,0.0,218,176.42586,112.11412,159.58055,115.59402,136
1.2.826.0.1.3680043.24673,2,135,None,None,3.875995,0.25,0.968999,0.0,218,175.00000,107.29992,162.19858,120.40821,137
1.2.826.0.1.3680043.24673,2,136,None,None,3.875995,0.25,0.968999,0.0,218,174.47909,115.76605,160.60837,114.43927,138
1.2.826.0.1.3680043.24673,2,137,None,None,3.875995,0.25,0.968999,0.0,218,172.00000,112.42653,162.00000,116.68616,139


In [112]:
bbox_df[bbox_df["is_flip"] == 1].index.unique()

Index(['1.2.826.0.1.3680043.11300', '1.2.826.0.1.3680043.11401',
       '1.2.826.0.1.3680043.1363', '1.2.826.0.1.3680043.18077',
       '1.2.826.0.1.3680043.20120', '1.2.826.0.1.3680043.21684',
       '1.2.826.0.1.3680043.23257', '1.2.826.0.1.3680043.23904',
       '1.2.826.0.1.3680043.24206', '1.2.826.0.1.3680043.26040',
       '1.2.826.0.1.3680043.27275', '1.2.826.0.1.3680043.3072',
       '1.2.826.0.1.3680043.32071', '1.2.826.0.1.3680043.4110',
       '1.2.826.0.1.3680043.4553', '1.2.826.0.1.3680043.4740',
       '1.2.826.0.1.3680043.6620'],
      dtype='object', name='StudyInstanceUID')

In [113]:
bbox_df.loc["1.2.826.0.1.3680043.11300"]

,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,300.37133,249.80261,25.21524,19.53686,147
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,290.68260,249.32587,32.99710,17.80456,148
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,303.37955,262.19738,28.40446,16.68531,161
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,298.92168,260.29052,30.00197,17.70948,162
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,298.00000,260.00000,31.00000,20.00000,163
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,298.00000,263.52056,27.58658,19.17591,164
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,255.24793,260.70805,59.56288,34.19252,169
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,259.00000,263.00000,57.57686,30.00000,170
1.2.826.0.1.3680043.11300,1,None,None,None,1.946768,0.513672,1.0,1.0,235,262.00000,260.00000,53.00000,32.00000,171


In [115]:
bbox_df[bbox_df["is_flip"] == 1]["axial_index"] = 235 - bbox_df[bbox_df["is_flip"] == 1]["slice_number"] - 1 + bbox_df[bbox_df["is_flip"] == 1]["start_slice_number"]

/var/folders/3l/2d1zfhx90z18_kvtkwwz9xtw0000gn/T/ipykernel_66965/2569038724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbox_df[bbox_df["is_flip"] == 1]["axial_index"] = 235 - bbox_df[bbox_df["is_flip"] == 1]["slice_number"] - 1 + bbox_df[bbox_df["is_flip"] == 1]["start_slice_number"]


for is_flip = 1

In [121]:
bbox_df.loc[bbox_df["is_flip"] == 1, "axial_index"] = bbox_df[bbox_df["is_flip"] == 1]["num_slices"].values - bbox_df[bbox_df["is_flip"] == 1]["slice_number"].values - 1 + bbox_df[bbox_df["is_flip"] == 1]["start_slice_number"].values
bbox_df.loc["1.2.826.0.1.3680043.11300"]

/Users/longyikim/anaconda3/envs/pytorch/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.11300,1,88,None,None,1.946768,0.513672,1.0,1.0,235,300.37133,249.80261,25.21524,19.53686,147
1.2.826.0.1.3680043.11300,1,87,None,None,1.946768,0.513672,1.0,1.0,235,290.68260,249.32587,32.99710,17.80456,148
1.2.826.0.1.3680043.11300,1,74,None,None,1.946768,0.513672,1.0,1.0,235,303.37955,262.19738,28.40446,16.68531,161
1.2.826.0.1.3680043.11300,1,73,None,None,1.946768,0.513672,1.0,1.0,235,298.92168,260.29052,30.00197,17.70948,162
1.2.826.0.1.3680043.11300,1,72,None,None,1.946768,0.513672,1.0,1.0,235,298.00000,260.00000,31.00000,20.00000,163
1.2.826.0.1.3680043.11300,1,71,None,None,1.946768,0.513672,1.0,1.0,235,298.00000,263.52056,27.58658,19.17591,164
1.2.826.0.1.3680043.11300,1,66,None,None,1.946768,0.513672,1.0,1.0,235,255.24793,260.70805,59.56288,34.19252,169
1.2.826.0.1.3680043.11300,1,65,None,None,1.946768,0.513672,1.0,1.0,235,259.00000,263.00000,57.57686,30.00000,170
1.2.826.0.1.3680043.11300,1,64,None,None,1.946768,0.513672,1.0,1.0,235,262.00000,260.00000,53.00000,32.00000,171


In [122]:
bbox_df.loc["1.2.826.0.1.3680043.11401"]

,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.11401,1,123,None,None,1.006004,0.447266,0.449951,1.0,382,124.00000,243.36378,42.00000,23.78645,259
1.2.826.0.1.3680043.11401,1,122,None,None,1.006004,0.447266,0.449951,1.0,382,123.00000,239.95442,44.00000,28.68178,260
1.2.826.0.1.3680043.11401,1,121,None,None,1.006004,0.447266,0.449951,1.0,382,122.00000,239.00000,45.00000,29.00000,261
1.2.826.0.1.3680043.11401,1,120,None,None,1.006004,0.447266,0.449951,1.0,382,121.00000,239.00000,47.00000,28.00000,262
1.2.826.0.1.3680043.11401,1,119,None,None,1.006004,0.447266,0.449951,1.0,382,120.00000,238.00000,49.00000,28.00000,263
1.2.826.0.1.3680043.11401,1,118,None,None,1.006004,0.447266,0.449951,1.0,382,120.00000,238.00000,51.00000,28.00000,264
1.2.826.0.1.3680043.11401,1,117,None,None,1.006004,0.447266,0.449951,1.0,382,119.00000,237.00000,53.00000,29.00000,265
1.2.826.0.1.3680043.11401,1,116,None,None,1.006004,0.447266,0.449951,1.0,382,118.00000,237.00000,54.00000,28.82815,266
1.2.826.0.1.3680043.11401,1,115,None,None,1.006004,0.447266,0.449951,1.0,382,117.00000,236.00000,56.00000,30.00000,267


In [123]:
bbox_df.loc[bbox_df["is_flip"] == 0, "axial_index"] = bbox_df[bbox_df["is_flip"] == 0]["slice_number"].values - bbox_df[bbox_df["is_flip"] == 0]["start_slice_number"].values
bbox_df.loc["1.2.826.0.1.3680043.10051"]

/Users/longyikim/anaconda3/envs/pytorch/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10051,1,132,None,None,2.461541,0.253906,0.625,0.0,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,1,133,None,None,2.461541,0.253906,0.625,0.0,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,1,134,None,None,2.461541,0.253906,0.625,0.0,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,1,135,None,None,2.461541,0.253906,0.625,0.0,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,1,136,None,None,2.461541,0.253906,0.625,0.0,272,214.00000,215.48637,27.00000,43.51363,137
1.2.826.0.1.3680043.10051,1,137,None,None,2.461541,0.253906,0.625,0.0,272,213.56364,216.71419,25.13455,47.92717,138
1.2.826.0.1.3680043.10051,1,138,None,None,2.461541,0.253906,0.625,0.0,272,209.00000,214.57591,36.06131,51.56834,139
1.2.826.0.1.3680043.10051,1,139,None,None,2.461541,0.253906,0.625,0.0,272,204.25455,212.24727,47.47636,57.71635,140
1.2.826.0.1.3680043.10051,1,140,None,None,2.461541,0.253906,0.625,0.0,272,205.00000,213.87342,45.00000,59.12658,141


In [140]:
bbox_df.loc["1.2.826.0.1.3680043.11300"]

,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.11300,1,88,259,312,1.946768,0.513672,1.0,1.0,235,300.37133,249.80261,25.21524,19.53686,147
1.2.826.0.1.3680043.11300,1,87,258,307,1.946768,0.513672,1.0,1.0,235,290.68260,249.32587,32.99710,17.80456,148
1.2.826.0.1.3680043.11300,1,74,270,317,1.946768,0.513672,1.0,1.0,235,303.37955,262.19738,28.40446,16.68531,161
1.2.826.0.1.3680043.11300,1,73,269,313,1.946768,0.513672,1.0,1.0,235,298.92168,260.29052,30.00197,17.70948,162
1.2.826.0.1.3680043.11300,1,72,270,313,1.946768,0.513672,1.0,1.0,235,298.00000,260.00000,31.00000,20.00000,163
1.2.826.0.1.3680043.11300,1,71,273,311,1.946768,0.513672,1.0,1.0,235,298.00000,263.52056,27.58658,19.17591,164
1.2.826.0.1.3680043.11300,1,66,277,285,1.946768,0.513672,1.0,1.0,235,255.24793,260.70805,59.56288,34.19252,169
1.2.826.0.1.3680043.11300,1,65,278,287,1.946768,0.513672,1.0,1.0,235,259.00000,263.00000,57.57686,30.00000,170
1.2.826.0.1.3680043.11300,1,64,276,288,1.946768,0.513672,1.0,1.0,235,262.00000,260.00000,53.00000,32.00000,171


In [136]:
bbox_df["sagittal_index"] = np.uint(np.floor(bbox_df["x"].values + bbox_df["width"].values / 2))
bbox_df.head()

/var/folders/3l/2d1zfhx90z18_kvtkwwz9xtw0000gn/T/ipykernel_66965/2299762219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbox_df["sagittal_index"] = np.uint(np.floor(bbox_df["x"].values + bbox_df["width"].values / 2))


,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10051,1,132,None,227,2.461541,0.253906,0.625,0.0,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,1,133,None,230,2.461541,0.253906,0.625,0.0,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,1,134,None,230,2.461541,0.253906,0.625,0.0,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,1,135,None,228,2.461541,0.253906,0.625,0.0,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,1,136,None,227,2.461541,0.253906,0.625,0.0,272,214.00000,215.48637,27.00000,43.51363,137


In [138]:
bbox_df["coronal_index"] = np.uint(np.floor(bbox_df["y"].values + bbox_df["height"].values / 2))
bbox_df.head()

/var/folders/3l/2d1zfhx90z18_kvtkwwz9xtw0000gn/T/ipykernel_66965/714918981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbox_df["coronal_index"] = np.uint(np.floor(bbox_df["y"].values + bbox_df["height"].values / 2))


,start_slice_number,axial_index,coronal_index,sagittal_index,aspect,pixel_spacing,z_spacing,is_flip,num_slices,x,y,width,height,slice_number
StudyInstanceUID,,,,,,,,,,,,,,
1.2.826.0.1.3680043.10051,1,132,226,227,2.461541,0.253906,0.625,0.0,272,219.27715,216.71419,17.30440,20.38517,133
1.2.826.0.1.3680043.10051,1,133,229,230,2.461541,0.253906,0.625,0.0,272,221.56460,216.71419,17.87844,25.24362,134
1.2.826.0.1.3680043.10051,1,134,234,230,2.461541,0.253906,0.625,0.0,272,216.82151,221.62546,27.00959,26.37454,135
1.2.826.0.1.3680043.10051,1,135,234,228,2.461541,0.253906,0.625,0.0,272,214.49455,215.48637,27.92726,37.51363,136
1.2.826.0.1.3680043.10051,1,136,237,227,2.461541,0.253906,0.625,0.0,272,214.00000,215.48637,27.00000,43.51363,137


In [141]:
bbox_df.to_csv('bbox_clean.csv')